In [ ]:
!mkdir -p ./kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d birdy654/cifake-real-and-ai-generated-synthetic-images

Dataset URL: https://www.kaggle.com/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images
License(s): other
 99% 104M/105M [00:07<00:00, 19.0MB/s]
100% 105M/105M [00:07<00:00, 14.5MB/s]


In [ ]:
!unzip /content/cifake-real-and-ai-generated-synthetic-images.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/train/REAL/4500 (10).jpg  
  inflating: /content/train/REAL/4500 (2).jpg  
  inflating: /content/train/REAL/4500 (3).jpg  
  inflating: /content/train/REAL/4500 (4).jpg  
  inflating: /content/train/REAL/4500 (5).jpg  
  inflating: /content/train/REAL/4500 (6).jpg  
  inflating: /content/train/REAL/4500 (7).jpg  
  inflating: /content/train/REAL/4500 (8).jpg  
  inflating: /content/train/REAL/4500 (9).jpg  
  inflating: /content/train/REAL/4500.jpg  
  inflating: /content/train/REAL/4501 (10).jpg  
  inflating: /content/train/REAL/4501 (2).jpg  
  inflating: /content/train/REAL/4501 (3).jpg  
  inflating: /content/train/REAL/4501 (4).jpg  
  inflating: /content/train/REAL/4501 (5).jpg  
  inflating: /content/train/REAL/4501 (6).jpg  
  inflating: /content/train/REAL/4501 (7).jpg  
  inflating: /content/train/REAL/4501 (8).jpg  
  inflating: /content/train/REAL/4501 (9).jpg  
  inflating: /content/train/REAL/4501.j

In [ ]:
!pip install --upgrade transformers

THE FILE IS IMPLEMENTATION OF RIGID FRAMEWORK   
**paper link**: https://arxiv.org/abs/2405.20112

In [ ]:
import os
import torch
import torch.nn.functional as F

from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

**LOADING MODEL**


In [ ]:
# Load model directly
from transformers import AutoImageProcessor, AutoModel

processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = AutoModel.from_pretrained("facebook/dinov2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
import torch
print (torch.cuda.is_available())

True


In [ ]:
model.eval().to("cuda")

Dinov2Model(
  (embeddings): Dinov2Embeddings(
    (patch_embeddings): Dinov2PatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): Dinov2Encoder(
    (layer): ModuleList(
      (0-11): 12 x Dinov2Layer(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attention): Dinov2SdpaAttention(
          (attention): Dinov2SdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): Dinov2SelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (layer_scale1): Dinov2LayerScale()
        (drop_pa

In [ ]:
!pip install torchvision

In [ ]:
import torchvision.transforms as transforms

In [ ]:
def load_image(image_path):
    # Load and preprocess the image using torchvision transforms
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Ensure the image size is correct
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822 , 0.4465], std=[0.2023, 0.1994, 0.2010]),
    ])
    image = transform(image)
    return image.unsqueeze(0).to("cuda")

In [ ]:
def add_noise (image, noise_level=0.05):
  noise= torch.randn(image.size(), device=image.device) * noise_level
  return image + noise



In [ ]:
def add_blur (image,sigma=1.0):
  transform= transforms.GaussianBlur(kernel_size=5, sigma=sigma)
  image_pil= transforms.ToPILImage()(image.cpu())
  blurred_image= transform(image_pil)
  return transforms.ToTensor()(blurred_image).to("cuda")

In [ ]:
def get_feature_dino (model,image):
  with torch.no_grad():
    image_features = model(image).last_hidden_state
  return image_features.mean(dim=1)

In [ ]:
def get_feature_resnet (model,image):
  with torch.no_grad():
    image_features = model(image)
  return image_features

In [ ]:
def cosine_sim (feat1, feat2):
  return F.cosine_similarity(feat1, feat2).item()

In [ ]:
def is_ai_resnet (image_path, model, threshold=0.95,noise_level=0.05):
  original= load_image(image_path)
  perturbed_image= add_noise(original,noise_level=noise_level)

  original_feature= get_feature_resnet(model, original)
  perturbed_feature= get_feature_resnet(model, perturbed_image)
  similarity= cosine_sim(original_feature, perturbed_feature)
  return similarity < threshold

In [ ]:
def is_ai_dino (image_path, model, threshold=0.95,noise_level=0.05):
  original= load_image(image_path)
  perturbed_image= add_noise(original,noise_level=noise_level)

  original_feature= get_feature_dino(model, original)
  perturbed_feature= get_feature_dino(model, perturbed_image)
  similarity= cosine_sim(original_feature, perturbed_feature)
  return similarity < threshold

In [ ]:
test_folder= "/content/test"
real= os.path.join(test_folder, "REAL")
ai= os.path.join(test_folder, "FAKE")

In [ ]:
true_labels= []
predicted_labels= []
image_count=0

In [ ]:

def classify_folder_dino (folder, label,model,threshold=0.95):
  global image_count
  for filename in os.listdir(folder):
    image_path= os.path.join(folder, filename)
    if os.path.isfile(image_path):
      is_ai_image= is_ai_dino(image_path, model, threshold)
      true_labels.append(label)
      predicted_labels.append(1 if is_ai_image else 0)
      image_count+=1
      if image_count % 100 == 0:
        print (f"Processed {image_count} images")

In [ ]:
def classify_folder_resnet (folder, label,model,threshold=0.95):
  global image_count
  for filename in os.listdir(folder):
    image_path= os.path.join(folder, filename)
    if os.path.isfile(image_path):
      is_ai_image= is_ai_resnet(image_path, model, threshold)
      true_labels.append(label)
      predicted_labels.append(1 if is_ai_image else 0)
      image_count+=1
      if image_count % 100 == 0:
        print (f"Processed {image_count} images")

In [ ]:
def tune_threshold_resnet (real_folder, fake_folder,model,noise_level=0.05):
  best_threshold= 0.95
  best_f1= 0
  thresholds= [0.80,0.85,0.90,0.92,0.95,0.97]
  for threshold in thresholds:
    global true_labels
    global predicted_labels
    true_labels= []
    predicted_labels= []
    classify_folder_resnet(real_folder, 0, model,threshold)
    classify_folder_resnet(fake_folder, 1, model, threshold)
    precision= precision_score(true_labels, predicted_labels)
    recall= recall_score(true_labels, predicted_labels)
    f1= f1_score(true_labels, predicted_labels)
    print (f"Threshold: {threshold}")
    print (f"Precision: {precision}")
    print (f"Recall: {recall}")
    print (f"F1 Score: {f1}")
    if f1 > best_f1:
      best_f1= f1
      best_threshold= threshold
  return best_threshold

In [ ]:
import torchvision.models as models

model2 = models.resnet18(pretrained=True)
model2 = model2.eval().to("cuda")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 142MB/s]


In [ ]:
best_threshold= tune_threshold_resnet(real, ai,model2,noise_level=0.05)

Processed 100 images
Processed 200 images
Processed 300 images
Processed 400 images
Processed 500 images
Processed 600 images
Processed 700 images
Processed 800 images
Processed 900 images
Processed 1000 images
Processed 1100 images
Processed 1200 images
Processed 1300 images
Processed 1400 images
Processed 1500 images
Processed 1600 images
Processed 1700 images
Processed 1800 images
Processed 1900 images
Processed 2000 images
Processed 2100 images
Processed 2200 images
Processed 2300 images
Processed 2400 images
Processed 2500 images
Processed 2600 images
Processed 2700 images
Processed 2800 images
Processed 2900 images
Processed 3000 images
Processed 3100 images
Processed 3200 images
Processed 3300 images
Processed 3400 images
Processed 3500 images
Processed 3600 images
Processed 3700 images
Processed 3800 images
Processed 3900 images
Processed 4000 images
Processed 4100 images
Processed 4200 images
Processed 4300 images
Processed 4400 images
Processed 4500 images
Processed 4600 imag

In [ ]:
print (best_threshold)

0.97


In [ ]:
def tune_threshold_resnet (real_folder, fake_folder,model,noise_level=0.05):
  best_threshold= 0.95
  best_f1= 0
  thresholds= [0.95,0.96,0.97,0.98,0.99,0.995,0.999]
  for threshold in thresholds:
    global true_labels
    global predicted_labels
    true_labels= []
    predicted_labels= []
    classify_folder_resnet(real_folder, 0, model,threshold)
    classify_folder_resnet(fake_folder, 1, model, threshold)
    precision= precision_score(true_labels, predicted_labels)
    recall= recall_score(true_labels, predicted_labels)
    f1= f1_score(true_labels, predicted_labels)
    print (f"Threshold: {threshold}")
    print (f"Precision: {precision}")
    print (f"Recall: {recall}")
    print (f"F1 Score: {f1}")
    if f1 > best_f1:
      best_f1= f1
      best_threshold= threshold
  return best_threshold

In [ ]:
best_threshold= tune_threshold_resnet(real, ai,model2,noise_level=0.05)

Processed 120100 images
Processed 120200 images
Processed 120300 images
Processed 120400 images
Processed 120500 images
Processed 120600 images
Processed 120700 images
Processed 120800 images
Processed 120900 images
Processed 121000 images
Processed 121100 images
Processed 121200 images
Processed 121300 images
Processed 121400 images
Processed 121500 images
Processed 121600 images
Processed 121700 images
Processed 121800 images
Processed 121900 images
Processed 122000 images
Processed 122100 images
Processed 122200 images
Processed 122300 images
Processed 122400 images
Processed 122500 images
Processed 122600 images
Processed 122700 images
Processed 122800 images
Processed 122900 images
Processed 123000 images
Processed 123100 images
Processed 123200 images
Processed 123300 images
Processed 123400 images
Processed 123500 images
Processed 123600 images
Processed 123700 images
Processed 123800 images
Processed 123900 images
Processed 124000 images
Processed 124100 images
Processed 124200